## Transfer Learning from CNN-LSTM


### Method 1 - 2 
- Data preprocess the same way for input
- Only train the second to last +last cnn layer, and freeze the rest


### Bench Marks
**Final Result for Random Forest trained on SOCAT**

Test Set RMSE: 30.56 | Whole Grid Rmse: 42.12


**Final Result for XGBoost trained on SOCAT**

Test Set RMSE:28.43698261274142 | Whole Grid Rmse:37.709863752151215

### Result
**[:-2],*[:-4],'../../models/transfer_CNN_LSTM_FINAL.h5'  BEST PERFORMING**


- resting and training again help lower the rmse score

In [13]:
tf.keras.backend.clear_session()

In [1]:
def inverse_scale_image_nfp(arr, df):
    """
    inverse_scale_image(arr, df):
    - inverses the pco2 scaling
    """
    df_tmp = df[df!=0.0]
    
    old_min = np.nanmin(df_tmp)
    old_max = np.nanmax(df_tmp)
    y_pred = arr*(old_max-old_min)/255+old_min
    
    tmp=np.nan_to_num(df[X_index][1:])
    y_true=np.expand_dims(tmp,axis=4)
    y_pred[y_true==0]=0
    return y_true,y_pred

In [2]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

import os
tf.random.set_seed(42)


sys.path.insert(0, '../../src')
from utils import df_to_xarray,read_xarray,inverse_scale_image, get_point_prediction

sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_image_reduced,preprocess_images_nfp

/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
2022-03-22 11:06:35.736312: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2022-03-22 11:06:40.756768: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-22 11:06:40.760784: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-22 11:06:40.830919: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-03-22 11:06:40.830972: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-22 11:06:40.877270: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-03-22 11:06:40.877337: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-0

In [4]:
import tensorflow.keras.backend as kb
import tensorflow as tf
from tensorflow.keras import backend as K

def custom_rmse(y_true, y_pred):
    """
    custom_rmse(y_true, y_pred)
    calculates root square mean value with focusing only on the ocean
    """
    y_pred = y_pred[(y_true != 0) & (y_true != 0.0)]
    y_true = y_true[(y_true != 0) & (y_true != 0.0)]
    
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    return K.sqrt(K.mean(tf.math.squared_difference(y_pred, y_true),axis= -1))

In [5]:
dir_name="../../data/data1"
val_dir_name="../../data/data2"

data,pco2 = preprocess_images_nfp(dir_name)
data_socat, pco2_socat = preprocess_images_nfp(dir_name, socat = True)

val_data,val_pco2 = preprocess_images_nfp(val_dir_name,"035")
val_data_socat,val_pco2_socat = preprocess_images_nfp(val_dir_name,"035",socat=True)



/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
ecCodes library not found using ['eccodes', 'libeccodes.so', 'libeccodes']
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [6]:
X_index=np.lib.stride_tricks.sliding_window_view(range(421),3) 

y=np.expand_dims(pco2_socat[X_index][1:],axis=4)
X=data_socat[X_index][:-1]

#validation data
val_y=np.expand_dims(val_pco2_socat[X_index][1:],axis=4)
val_X=val_data_socat[X_index][:-1]

print(X.shape, y.shape)

INPUT_SHAPE = X.shape[0]

(418, 3, 180, 360, 6) (418, 3, 180, 360, 1)


In [7]:
from tensorflow.keras.layers import Dense, Reshape, Conv2D, MaxPool2D , Flatten, Input

base_model = tf.keras.models.load_model('../../models/base_CNN_LSTM_new.h5', custom_objects={'custom_rmse':custom_rmse})


2022-03-22 11:06:55.995941: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 11:06:55.996352: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-22 11:06:55.998190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-03-22 11:06:55.998292: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-22 11:06:55.998340: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Succ

In [8]:
for index, layer in enumerate(base_model.layers):
    print(index, layer)

0 <tensorflow.python.keras.layers.convolutional_recurrent.ConvLSTM2D object at 0x1554baa575e0>
1 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x15550d0e1f70>
2 <tensorflow.python.keras.layers.convolutional_recurrent.ConvLSTM2D object at 0x1554b9691850>
3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x1554b96d4ca0>
4 <tensorflow.python.keras.layers.convolutional_recurrent.ConvLSTM2D object at 0x1554bab7bdf0>
5 <tensorflow.python.keras.layers.convolutional.Conv3D object at 0x15550d0ec130>


In [9]:
for layer in base_model.layers[:-4]:
    layer.trainable = False
    
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 3, 180, 360, 32)   121728    
_________________________________________________________________
batch_normalization (BatchNo (None, 3, 180, 360, 32)   128       
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 3, 180, 360, 32)   73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 180, 360, 32)   128       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 3, 180, 360, 32)   8320      
_________________________________________________________________
conv3d (Conv3D)              (None, 3, 180, 360, 1)    865       
Total params: 205,025
Trainable params: 83,105
Non-trainable params: 121,920
_____________________________________________

In [10]:
# inputs = layers.Input(shape = INPUT_SHAPE, name='inputLayer')
# x = base_model(inputs)
# x = layers.GlobalAveragePooling2D(name='poolingLayer')(x)
# x = layers.Dense(101, name='outputLayer')(x)
# outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x)

In [14]:
base_model.compile(
    loss=custom_rmse, optimizer=keras.optimizers.Adam(learning_rate=1e-4),
)

model_path="../../models/transfer_CNN_LSTM.h5"

early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

epochs = 50
batch_size = 16

In [15]:
history = base_model.fit(
    X,
    y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X, y),
    callbacks=callbacks,
)

Epoch 1/50
27/27 [==============================] - 41s 1s/step - loss: 58.0616 - val_loss: 39.2514
Epoch 2/50
27/27 [==============================] - 36s 1s/step - loss: 42.4693 - val_loss: 31.9344
Epoch 3/50
27/27 [==============================] - 36s 1s/step - loss: 34.0728 - val_loss: 27.7215
Epoch 4/50
27/27 [==============================] - 36s 1s/step - loss: 29.0866 - val_loss: 28.4075
Epoch 5/50
27/27 [==============================] - 36s 1s/step - loss: 25.3535 - val_loss: 29.3416
Epoch 6/50
27/27 [==============================] - 36s 1s/step - loss: 23.1330 - val_loss: 29.4942
Epoch 7/50
27/27 [==============================] - 37s 1s/step - loss: 20.9493 - val_loss: 27.4018
Epoch 8/50
27/27 [==============================] - 36s 1s/step - loss: 19.6307 - val_loss: 24.6662
Epoch 9/50
27/27 [==============================] - 36s 1s/step - loss: 19.0597 - val_loss: 23.0448
Epoch 10/50
27/27 [==============================] - 36s 1s/step - loss: 17.5716 - val_loss: 21.0751

KeyboardInterrupt: 

In [16]:
from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=80)

plt.plot(history.history["val_loss"])
plt.ylabel('val_loss')
plt.xlabel('epoch')

NameError: name 'history' is not defined

<Figure size 640x480 with 0 Axes>

### Assessing Performance

In [17]:
best_model = tf.keras.models.load_model('../../models/transfer_CNN_LSTM.h5', custom_objects={'custom_rmse':custom_rmse})

X_index=np.lib.stride_tricks.sliding_window_view(range(421),3) 

y=np.expand_dims(pco2_socat[X_index][1:],axis=4)
X=data_socat[X_index][:-1]

predicted_frames=best_model.predict(X,verbose=1)

14/14 [==============================] - 11s 709ms/step


In [18]:
predicted_frames[y==0]=0.0

In [19]:
custom_rmse(y,predicted_frames)

<tf.Tensor: shape=(), dtype=float32, numpy=14.097132>

In [ ]:
import matplotlib.colors as mcolors
figure, axis = plt.subplots(2, 2,figsize=(12, 6))

d = predicted_frames - y


norm = mcolors.TwoSlopeNorm(vmin=d.min(), vmax = d.max(), vcenter=0)


img=axis[0][0].imshow(np.flipud(predicted_frames[0][1]),cmap="coolwarm", interpolation="nearest")
axis[0][0].set_title("prediction")
plt.colorbar(img,ax=axis)

img1=axis[0][1].imshow(np.flipud(y[0][1]),cmap="coolwarm", interpolation="nearest")
axis[0][1].set_title("true")

diff=np.flipud(np.squeeze(predicted_frames[0][1]-y[0][1]))
img2=axis[1][0].imshow(diff,cmap="RdBu", interpolation="nearest",norm=norm)
axis[1][0].set_title("residual")
plt.colorbar(img2,ax=axis)


img2=axis[1][1].imshow(np.flipud(X[0][1][:,:,5]),cmap="coolwarm", interpolation="nearest")
axis[1][1].set_title("input: previous pco2")
plt.show()

### Inverse

In [20]:
#chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2_tmp = read_xarray(val_dir_name,"035")

chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2_tmp = read_xarray(dir_name)
y_true,y_pred=inverse_scale_image_nfp(predicted_frames,pco2_tmp.pCO2_socat.data)  

In [21]:
y_pred_tmp = y_pred[:,:1]
y_true_tmp = y_true[:,:1]

# y_pred_tmp = y_pred
# y_true_tmp = y_true


y_pred_tmp = y_pred_tmp[y_true_tmp != 0.0]
y_true_tmp = y_true_tmp[y_true_tmp != 0.0]


print("Scaled back socat RMSE score:")
np.sqrt(np.mean((y_true_tmp-y_pred_tmp)**2))

Scaled back socat RMSE score:


58.39261190784995

In [22]:

custom_rmse(y_pred_tmp,y_true_tmp)




<tf.Tensor: shape=(), dtype=float64, numpy=58.39261190784995>

### full data set


In [23]:

y=np.expand_dims(pco2[X_index][1:],axis=4)
X=data[X_index][:-1]

predicted_frames=best_model.predict(X,verbose=1)
predicted_frames[y==0]=0.0

chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2_tmp = read_xarray(dir_name)
y_true,y_pred=inverse_scale_image_nfp(predicted_frames,pco2_tmp.pCO2.data)  

14/14 [==============================] - 10s 751ms/step


In [24]:
y_pred_tmp = y_pred[:,:1]
y_true_tmp = y_true[:,:1]

custom_rmse(y_pred_tmp,y_true_tmp)



<tf.Tensor: shape=(), dtype=float64, numpy=63.600287862634175>